In [ ]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, r2_score

# 📌 Ensure the Random Forest model is downloaded first
random_forest_model_path = "models/random_forest.pkl"

if not os.path.exists(random_forest_model_path):
    print("Random Forest model not found. Running random_forest_download.py...")
    subprocess.run(["python", "random_forest_download.py"], check=True)
    print("Random Forest model downloaded successfully!")

# Function to evaluate a model
def evaluate_model(model_name, model_path, X_test_path, y_test_path, is_lstm=False):
    print(f"\nEvaluating {model_name}...")

    # Load test data
    if is_lstm:
        X_test = np.load(X_test_path)
        y_test = np.load(y_test_path)
        model = load_model(model_path)
    else:
        X_test = joblib.load(X_test_path)
        y_test = joblib.load(y_test_path)
        model = joblib.load(model_path)

    # Make predictions
    predictions = model.predict(X_test).flatten()

    # Compute evaluation metrics
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)

    print(f"{model_name} - MAE: {mae}")
    print(f"{model_name} - R² Score: {r2}")

    # Plot actual vs. predicted values
    plt.figure(figsize=(6,4))
    plt.scatter(y_test, predictions, alpha=0.5)
    plt.xlabel("Actual Prices")
    plt.ylabel("Predicted Prices")
    plt.title(f"{model_name} - Actual vs. Predicted")
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')  # Identity line
    plt.show()

# Run evaluations
evaluate_model(
    model_name="Random Forest",
    model_path=random_forest_model_path,
    X_test_path="test_data/RandomForest/X_test_rf.pkl",
    y_test_path="test_data/RandomForest/y_test_rf.pkl",
)

evaluate_model(
    model_name="Linear Regression",
    model_path="models/linear_regression.pkl",
    X_test_path="test_data/LinearRegression/X_test_lr.pkl",
    y_test_path="test_data/LinearRegression/y_test_lr.pkl",
)

evaluate_model(
    model_name="LSTM",
    model_path="models/lstm_model.h5",
    X_test_path="test_data/LSTM/X_test_lstm.npy",
    y_test_path="test_data/LSTM/y_test_lstm.npy",
    is_lstm=True,
)

